# 00 — From Transistors to Logic Gates

**Module 06: Integrated Circuits**

---

We have spent five modules understanding individual semiconductor devices: diodes, BJTs, MOSFETs. Now we make the leap that changed civilization: **combining transistors into logic gates**, and logic gates into everything from calculators to smartphones.

This notebook covers the bridge between analog transistor circuits and digital logic. By the end, you will:
- Understand how a MOSFET acts as a digital switch (HIGH/LOW, 1/0)
- Build a CMOS inverter from complementary NMOS + PMOS transistors
- Construct a NAND gate from discrete MOSFETs and verify its truth table
- Understand why NAND is the universal gate
- See the conceptual path from gates to flip-flops to processors
- Appreciate Moore's Law and what it means physically

## Concept — Digital Logic: HIGH and LOW

In analog electronics, voltages can be anything: 1.23V, 3.87V, 0.01V. In digital electronics, we simplify to just **two states**:

| State | Name | Typical Voltage (5V logic) | Typical Voltage (3.3V logic) |
|-------|------|---------------------------|-----------------------------|
| 1 | HIGH | > 3.5V (close to VCC) | > 2.4V (close to VCC) |
| 0 | LOW | < 1.5V (close to GND) | < 0.8V (close to GND) |

The exact thresholds depend on the **logic family** (more on this in the next notebook), but the principle is always the same: we throw away analog precision in exchange for **noise immunity**. A signal can be corrupted by 0.5V of noise and still be read correctly.

### The fundamental building block: the switch

A MOSFET is a voltage-controlled switch:
- **NMOS**: Gate HIGH = switch ON (drain-source conducts), Gate LOW = switch OFF
- **PMOS**: Gate LOW = switch ON, Gate HIGH = switch OFF (opposite!)

```
NMOS as switch:                PMOS as switch:

  Gate = HIGH --> ON            Gate = LOW  --> ON
  Gate = LOW  --> OFF           Gate = HIGH --> OFF

   D                             S
   |                             |
   | (conducts when              | (conducts when
   |  Vgs > Vth)                 |  Vgs < -|Vth|)
   |                             |
   S                             D
```

This complementary behavior is the key insight behind CMOS logic.

## Concept — The CMOS Inverter (NOT Gate)

The simplest logic gate uses one NMOS and one PMOS transistor:

```
        VCC (5V)
         |
       |--+  PMOS (Q1)
 IN ---+       
       |--+  
         |------- OUT
       |--+
 IN ---+     NMOS (Q2)
       |--+
         |
        GND
```

### How it works:

**When IN = HIGH (VCC):**
- NMOS gate is HIGH relative to source (GND) --> Vgs > Vth --> NMOS **ON**
- PMOS gate is HIGH relative to source (VCC) --> Vgs = 0 --> PMOS **OFF**
- Output connects to GND through NMOS --> **OUT = LOW**

**When IN = LOW (GND):**
- NMOS gate is LOW --> Vgs = 0 --> NMOS **OFF**
- PMOS gate is LOW relative to source (VCC) --> Vgs = -VCC --> PMOS **ON**
- Output connects to VCC through PMOS --> **OUT = HIGH**

| IN | PMOS | NMOS | OUT |
|----|------|------|-----|
| 0  | ON   | OFF  | 1   |
| 1  | OFF  | ON   | 0   |

This is a **NOT gate** (inverter): output is always the opposite of input.

### The critical advantage: no static power consumption

In every stable state, one transistor is ON and the other is OFF. There is **never a direct path from VCC to GND**. Current only flows during the brief switching transition. This is why CMOS dominates: a chip with billions of transistors does not melt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# CMOS Inverter Transfer Characteristic: Vout vs Vin
# Using simplified MOSFET models

VCC = 5.0
Vtn = 1.0   # NMOS threshold
Vtp = -1.0  # PMOS threshold (negative)
kn = 2.0    # NMOS transconductance parameter (mA/V^2)
kp = 1.0    # PMOS transconductance parameter (mA/V^2)

Vin = np.linspace(0, VCC, 1000)
Vout = np.zeros_like(Vin)

for i, vi in enumerate(Vin):
    # Solve for Vout where NMOS current = PMOS current
    # Simplified: use a smooth tanh approximation for the transfer curve
    # Real CMOS inverter has a sharp transition near VCC/2
    midpoint = VCC / 2
    steepness = 8.0 / VCC  # Sharper for higher VCC
    Vout[i] = VCC / 2 * (1 - np.tanh(steepness * (vi - midpoint)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Transfer characteristic
ax1.plot(Vin, Vout, 'b-', linewidth=2.5)
ax1.set_xlabel('Input Voltage (V)', fontsize=12)
ax1.set_ylabel('Output Voltage (V)', fontsize=12)
ax1.set_title('CMOS Inverter Transfer Characteristic', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, VCC)
ax1.set_ylim(-0.2, VCC + 0.2)

# Shade logic regions
ax1.axhspan(3.5, 5.2, alpha=0.15, color='green', label='Output HIGH region')
ax1.axhspan(-0.2, 1.5, alpha=0.15, color='red', label='Output LOW region')
ax1.axvspan(0, 1.5, alpha=0.1, color='blue')
ax1.axvspan(3.5, 5.0, alpha=0.1, color='orange')
ax1.text(0.5, 4.7, 'IN=LOW\nOUT=HIGH', fontsize=9, ha='center', color='green')
ax1.text(4.3, 0.3, 'IN=HIGH\nOUT=LOW', fontsize=9, ha='center', color='red')

# Mark the switching threshold
ax1.axvline(x=VCC/2, color='gray', linestyle='--', alpha=0.5)
ax1.text(VCC/2 + 0.1, 2.5, f'Switching\nthreshold\n~{VCC/2:.1f}V', fontsize=9, color='gray')
ax1.legend(fontsize=10, loc='center right')

# Current during switching
# Short-circuit current peaks during transition
I_peak = 0.5 * kn * (VCC/2 - Vtn)**2  # rough peak current
I_short = np.zeros_like(Vin)
for i, vi in enumerate(Vin):
    # Both transistors partially on in the transition region
    if vi > Vtn and vi < (VCC + Vtp):
        n_frac = min((vi - Vtn) / (VCC/2 - Vtn), 1.0)
        p_frac = min((VCC + Vtp - vi) / (VCC/2 + Vtp), 1.0)
        I_short[i] = I_peak * n_frac * p_frac

ax2.plot(Vin, I_short, 'r-', linewidth=2.5)
ax2.fill_between(Vin, 0, I_short, alpha=0.3, color='red')
ax2.set_xlabel('Input Voltage (V)', fontsize=12)
ax2.set_ylabel('Short-Circuit Current (mA)', fontsize=12)
ax2.set_title('CMOS Inverter: Current During Switching', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.text(2.5, I_peak * 0.8, 'Current flows ONLY\nduring transition!', fontsize=11,
         ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print('Key observation: In both stable states (IN=0 or IN=5V), current is essentially ZERO.')
print('Power is consumed only during switching transitions.')
print('This is why CMOS can have billions of transistors without melting.')

## Concept — NAND Gate from Transistors

The NAND gate is the most important gate in digital logic. It uses **2 NMOS in series** (pull-down) and **2 PMOS in parallel** (pull-up):

```
              VCC
             /   \
          P1       P2      (PMOS in PARALLEL)
         |--+     +--|
   A ----|  |     |  |---- B
         |--+     +--|
             \   /
              +----------- OUT
              |
            |--+
      A ----|     N1       (NMOS in SERIES)
            |--+
              |
            |--+
      B ----|     N2
            |--+
              |
             GND
```

### Logic analysis:

**Output = LOW requires** a path from OUT to GND:
- Both NMOS must be ON --> both A and B must be HIGH
- This is the ONLY case where output is LOW

**Output = HIGH requires** a path from OUT to VCC:
- At least one PMOS must be ON --> at least one input must be LOW

| A | B | P1 (PMOS) | P2 (PMOS) | N1 (NMOS) | N2 (NMOS) | Path to VCC? | Path to GND? | OUT |
|---|---|-----------|-----------|-----------|-----------|-------------|-------------|-----|
| 0 | 0 | ON        | ON        | OFF       | OFF       | YES (both)  | NO          | 1   |
| 0 | 1 | ON        | OFF       | OFF       | ON        | YES (P1)    | NO          | 1   |
| 1 | 0 | OFF       | ON        | ON        | OFF       | YES (P2)    | NO          | 1   |
| 1 | 1 | OFF       | OFF       | ON        | ON        | NO          | YES (both)  | 0   |

This is the **NAND** truth table: output is LOW only when **both** inputs are HIGH.

In [ ]:
import numpy as np

# Truth table generator for common logic gates

def print_truth_table(gate_name, func):
    """Print a formatted truth table for a 2-input gate."""
    print(f'\n  {gate_name} Gate Truth Table')
    print(f'  {"="*30}')
    print(f'  {"A":^5} | {"B":^5} | {"OUT":^5}')
    print(f'  {"-"*5}-+-{"-"*5}-+-{"-"*5}')
    for a in [0, 1]:
        for b in [0, 1]:
            out = func(a, b)
            print(f'  {a:^5} | {b:^5} | {out:^5}')

def print_not_table():
    print(f'\n  NOT Gate (Inverter) Truth Table')
    print(f'  {"="*20}')
    print(f'  {"IN":^5} | {"OUT":^5}')
    print(f'  {"-"*5}-+-{"-"*5}')
    for a in [0, 1]:
        print(f'  {a:^5} | {1-a:^5}')

# Define gate functions
gates = {
    'AND':  lambda a, b: a & b,
    'OR':   lambda a, b: a | b,
    'NAND': lambda a, b: 1 - (a & b),
    'NOR':  lambda a, b: 1 - (a | b),
    'XOR':  lambda a, b: a ^ b,
    'XNOR': lambda a, b: 1 - (a ^ b),
}

print_not_table()
for name, func in gates.items():
    print_truth_table(name, func)

print('\n\nKey observation: NAND is AND followed by NOT.')
print('NOR is OR followed by NOT.')
print('Both NAND and NOR are "universal gates" -- you can build ANY logic function from just one type.')

## Concept — NOR Gate from Transistors

The NOR gate is the mirror image of NAND: **2 PMOS in series** (pull-up) and **2 NMOS in parallel** (pull-down):

```
              VCC
               |
            |--+
      A ----|     P1       (PMOS in SERIES)
            |--+
               |
            |--+
      B ----|     P2
            |--+
               |
               +----------- OUT
              / \
          N1       N2      (NMOS in PARALLEL)
         |--+     +--|
   A ----|  |     |  |---- B
         |--+     +--|
              \ /
              GND
```

| A | B | OUT |
|---|---|-----|
| 0 | 0 | 1   |
| 0 | 1 | 0   |
| 1 | 0 | 0   |
| 1 | 1 | 0   |

Output is HIGH **only** when **both** inputs are LOW.

### Why NAND won over NOR

Both NAND and NOR are universal gates, but NAND is preferred because:
1. **PMOS transistors are ~2-3x slower** than NMOS (holes have lower mobility than electrons)
2. In NAND, the slow PMOS devices are in **parallel** (faster) while fast NMOS are in series
3. In NOR, the slow PMOS devices are in **series** (even slower) while fast NMOS are in parallel
4. NAND gates require fewer transistors for most practical logic functions

This is why most logic families (74HC00, etc.) are built around NAND gates.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Visualize which transistors are ON/OFF for each NAND input combination

fig, axes = plt.subplots(1, 4, figsize=(18, 6))

input_combos = [(0, 0), (0, 1), (1, 0), (1, 1)]
nand_outputs = [1, 1, 1, 0]

for idx, (a, b) in enumerate(input_combos):
    ax = axes[idx]
    out = nand_outputs[idx]
    
    # PMOS states (ON when input is LOW)
    p1_on = (a == 0)
    p2_on = (b == 0)
    # NMOS states (ON when input is HIGH)
    n1_on = (a == 1)
    n2_on = (b == 1)
    
    # Draw VCC rail
    ax.plot([0.2, 0.8], [0.95, 0.95], 'r-', linewidth=3)
    ax.text(0.5, 0.98, 'VCC', ha='center', fontsize=10, fontweight='bold', color='red')
    
    # Draw GND rail
    ax.plot([0.2, 0.8], [0.05, 0.05], 'k-', linewidth=3)
    ax.text(0.5, 0.01, 'GND', ha='center', fontsize=10, fontweight='bold')
    
    # PMOS P1 (parallel)
    color_p1 = '#4CAF50' if p1_on else '#BDBDBD'
    rect_p1 = patches.FancyBboxPatch((0.1, 0.72), 0.3, 0.15,
                                      boxstyle='round,pad=0.02',
                                      facecolor=color_p1, edgecolor='black', linewidth=1.5)
    ax.add_patch(rect_p1)
    ax.text(0.25, 0.795, f'P1\n{"ON" if p1_on else "OFF"}', ha='center', va='center', fontsize=8, fontweight='bold')
    
    # PMOS P2 (parallel)
    color_p2 = '#4CAF50' if p2_on else '#BDBDBD'
    rect_p2 = patches.FancyBboxPatch((0.6, 0.72), 0.3, 0.15,
                                      boxstyle='round,pad=0.02',
                                      facecolor=color_p2, edgecolor='black', linewidth=1.5)
    ax.add_patch(rect_p2)
    ax.text(0.75, 0.795, f'P2\n{"ON" if p2_on else "OFF"}', ha='center', va='center', fontsize=8, fontweight='bold')
    
    # Connect PMOS to VCC and output
    ax.plot([0.25, 0.25], [0.87, 0.95], 'r-' if p1_on else 'k:', linewidth=2 if p1_on else 1)
    ax.plot([0.75, 0.75], [0.87, 0.95], 'r-' if p2_on else 'k:', linewidth=2 if p2_on else 1)
    ax.plot([0.25, 0.25], [0.72, 0.62], 'r-' if p1_on else 'k:', linewidth=2 if p1_on else 1)
    ax.plot([0.75, 0.75], [0.72, 0.62], 'r-' if p2_on else 'k:', linewidth=2 if p2_on else 1)
    
    # Output node
    ax.plot([0.25, 0.75], [0.62, 0.62], 'k-', linewidth=2)
    ax.plot([0.5, 0.5], [0.62, 0.55], 'k-', linewidth=2)
    out_color = 'red' if out == 1 else 'blue'
    ax.plot(0.9, 0.59, 'o', color=out_color, markersize=15)
    ax.text(0.9, 0.59, str(out), ha='center', va='center', fontsize=11, fontweight='bold', color='white')
    ax.plot([0.5, 0.9], [0.59, 0.59], 'k-', linewidth=2)
    ax.text(0.9, 0.52, 'OUT', ha='center', fontsize=8)
    
    # NMOS N1 (series - top)
    color_n1 = '#2196F3' if n1_on else '#BDBDBD'
    rect_n1 = patches.FancyBboxPatch((0.3, 0.35), 0.4, 0.15,
                                      boxstyle='round,pad=0.02',
                                      facecolor=color_n1, edgecolor='black', linewidth=1.5)
    ax.add_patch(rect_n1)
    ax.text(0.5, 0.425, f'N1\n{"ON" if n1_on else "OFF"}', ha='center', va='center', fontsize=8, fontweight='bold')
    
    # NMOS N2 (series - bottom)
    color_n2 = '#2196F3' if n2_on else '#BDBDBD'
    rect_n2 = patches.FancyBboxPatch((0.3, 0.12), 0.4, 0.15,
                                      boxstyle='round,pad=0.02',
                                      facecolor=color_n2, edgecolor='black', linewidth=1.5)
    ax.add_patch(rect_n2)
    ax.text(0.5, 0.195, f'N2\n{"ON" if n2_on else "OFF"}', ha='center', va='center', fontsize=8, fontweight='bold')
    
    # Connect NMOS
    ax.plot([0.5, 0.5], [0.50, 0.55], 'b-' if (n1_on and n2_on) else 'k:', linewidth=2 if (n1_on and n2_on) else 1)
    ax.plot([0.5, 0.5], [0.27, 0.35], 'b-' if (n1_on and n2_on) else 'k:', linewidth=2 if (n1_on and n2_on) else 1)
    ax.plot([0.5, 0.5], [0.05, 0.12], 'b-' if (n1_on and n2_on) else 'k:', linewidth=2 if (n1_on and n2_on) else 1)
    
    ax.set_xlim(0, 1.05)
    ax.set_ylim(0, 1.05)
    ax.set_title(f'A={a}, B={b}  -->  OUT={out}', fontsize=12, fontweight='bold',
                 color='green' if out == 1 else 'red')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

fig.suptitle('CMOS NAND Gate: Transistor States for Each Input Combination',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print('Green boxes = transistor ON (conducting)')
print('Gray boxes = transistor OFF (blocking)')
print('Output is LOW only when BOTH NMOS are ON (A=1, B=1), creating path to GND.')

## The Material Science Why — CMOS and the Silicon Dioxide Gate

CMOS logic works because of the **MOSFET** structure we studied in Module 04. Here is why the physics matters for digital logic:

### The Gate Oxide is the Key

The gate of a MOSFET is insulated from the channel by a thin layer of SiO2 (or high-k dielectric in modern chips). This means:
- **Zero DC gate current** -- the gate is a capacitor, not a resistor
- One gate output can drive many gate inputs without current loading
- The only current needed is to charge/discharge the gate capacitance during switching

### Complementary Pairs

CMOS requires both N-type and P-type MOSFETs on the same chip:
- **NMOS**: built in a P-type substrate, N-type source/drain regions
- **PMOS**: built in an N-type well, P-type source/drain regions

```
Cross-section of CMOS inverter on silicon:

    PMOS (in N-well)              NMOS (in P-substrate)
  |  S   G   D  |             |  D   G   S  |
  | P+  |===| P+ |           | N+  |===| N+ |
  |    N-well     |           |   P-substrate  |
  |_______________|___________|________________|
              P-type substrate
```

### Scaling: Why Smaller is Better

The switching speed of a CMOS gate is limited by the time to charge/discharge gate capacitances:
- **Smaller transistors** = less capacitance = faster switching
- **Thinner gate oxide** = stronger field = faster channel formation
- **Shorter channel** = less distance for electrons to travel

This is the physical basis of Moore's Law: shrinking transistors makes them both faster and cheaper per transistor.

## Concept — NAND is Universal

Any Boolean function can be built from NAND gates alone. Here is how to build every basic gate from NANDs:

```
NOT from NAND:     Tie both inputs together
  A ---+
       |---[NAND]--- NOT(A)
  A ---+

AND from NAND:     NAND followed by NOT (which is another NAND)
  A ---[NAND]---[NOT]--- A AND B
  B ---|

OR from NAND:      De Morgan's: A OR B = NOT(NOT(A) AND NOT(B))
  A ---[NOT]---+
               |---[NAND]--- A OR B
  B ---[NOT]---+
```

This universality is why digital design tools can synthesize any circuit using just one gate type, simplifying manufacturing enormously.

In [ ]:
# Demonstrate NAND universality: build all gates from NAND

def NAND(a, b):
    return 1 - (a & b)

def NOT_from_NAND(a):
    return NAND(a, a)

def AND_from_NAND(a, b):
    return NOT_from_NAND(NAND(a, b))

def OR_from_NAND(a, b):
    return NAND(NOT_from_NAND(a), NOT_from_NAND(b))

def XOR_from_NAND(a, b):
    nand_ab = NAND(a, b)
    return NAND(NAND(a, nand_ab), NAND(b, nand_ab))

print('Verifying NAND universality -- all gates built from NAND only:')
print()

print(f'{"A":>3} {"B":>3} | {"NOT A":>5} | {"AND":>3} {"OR":>3} {"XOR":>3} {"NAND":>4}')
print('-' * 45)

all_correct = True
for a in [0, 1]:
    for b in [0, 1]:
        not_a = NOT_from_NAND(a)
        and_ab = AND_from_NAND(a, b)
        or_ab = OR_from_NAND(a, b)
        xor_ab = XOR_from_NAND(a, b)
        nand_ab = NAND(a, b)
        
        # Verify against direct computation
        assert not_a == (1 - a), f'NOT failed for {a}'
        assert and_ab == (a & b), f'AND failed for {a},{b}'
        assert or_ab == (a | b), f'OR failed for {a},{b}'
        assert xor_ab == (a ^ b), f'XOR failed for {a},{b}'
        
        print(f'{a:>3} {b:>3} | {not_a:>5} | {and_ab:>3} {or_ab:>3} {xor_ab:>3} {nand_ab:>4}')

print()
print('All gates verified correct! NAND truly is universal.')
print()
print('NAND count to build each gate:')
print('  NOT:  1 NAND gate')
print('  AND:  2 NAND gates')
print('  OR:   3 NAND gates')
print('  XOR:  4 NAND gates')

## Concept — From Gates to Processors

Here is the conceptual hierarchy from a single transistor to a CPU:

```
Transistor (MOSFET)
  --> Logic Gate (NAND, NOR, NOT)
    --> Combinational Logic (adder, multiplexer, decoder)
      --> Sequential Logic (flip-flop, register, counter)
        --> Functional Units (ALU, register file, control unit)
          --> Processor (CPU, GPU, microcontroller)
            --> System-on-Chip (CPU + memory + I/O)
```

### The flip-flop: memory from gates

Two cross-coupled NAND gates create a **set-reset (SR) latch** -- the simplest memory element. It can store one bit of information:

```
  S ---[NAND]--+---> Q
          |    |
          +----+
          |    |
  R ---[NAND]--+---> Q' (not Q)
```

Add a clock signal and you get a **D flip-flop** -- the building block of all registers and memory.

### The numbers are staggering

| Year | Processor | Transistor Count |
|------|-----------|------------------|
| 1971 | Intel 4004 | 2,300 |
| 1978 | Intel 8086 | 29,000 |
| 1993 | Pentium | 3,100,000 |
| 2006 | Core 2 Duo | 291,000,000 |
| 2015 | Skylake | 1,750,000,000 |
| 2022 | Apple M2 Ultra | 134,000,000,000 |

Every one of those transistors is a MOSFET. Every logic gate is a CMOS pair. The principles we study in this notebook scale from 2 transistors to 134 billion.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Moore's Law: transistor count over time

# Historical data points (year, transistor count, name)
processors = [
    (1971, 2300, 'Intel 4004'),
    (1974, 4500, '8080'),
    (1978, 29000, '8086'),
    (1982, 134000, '80286'),
    (1985, 275000, '80386'),
    (1989, 1200000, '80486'),
    (1993, 3100000, 'Pentium'),
    (1997, 7500000, 'Pentium II'),
    (1999, 9500000, 'Pentium III'),
    (2000, 42000000, 'Pentium 4'),
    (2006, 291000000, 'Core 2 Duo'),
    (2008, 731000000, 'Core i7 (1st)'),
    (2012, 1400000000, 'Core i7 (3rd)'),
    (2015, 1750000000, 'Skylake'),
    (2019, 8000000000, 'Apple A13'),
    (2021, 57000000000, 'Apple M1 Ultra'),
    (2023, 134000000000, 'Apple M2 Ultra'),
]

years = np.array([p[0] for p in processors])
counts = np.array([p[1] for p in processors])
names = [p[2] for p in processors]

# Moore's Law prediction: doubling every 2 years from 1971
years_pred = np.linspace(1971, 2025, 100)
moores_law = 2300 * 2 ** ((years_pred - 1971) / 2)

fig, ax = plt.subplots(figsize=(14, 7))

# Moore's Law line
ax.semilogy(years_pred, moores_law, 'r--', linewidth=2, alpha=0.7,
            label="Moore's Law (doubling every 2 years)")

# Actual data
ax.semilogy(years, counts, 'bo-', markersize=8, linewidth=1.5,
            label='Actual transistor counts')

# Label key processors
for i in [0, 4, 7, 10, 14, 16]:
    ax.annotate(names[i], xy=(years[i], counts[i]),
                xytext=(years[i] - 3, counts[i] * 4),
                fontsize=8, ha='center',
                arrowprops=dict(arrowstyle='->', color='black', lw=0.8))

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Transistor Count', fontsize=12)
ax.set_title("Moore's Law: Transistor Count Over Time", fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='lower right')
ax.grid(True, alpha=0.3, which='both')
ax.set_xlim(1970, 2025)
ax.set_ylim(1e3, 1e12)

# Add annotations for process nodes
ax.annotate('10 um process', xy=(1971, 5e3), fontsize=8, color='gray')
ax.annotate('1 um', xy=(1985, 5e5), fontsize=8, color='gray')
ax.annotate('100 nm', xy=(2000, 5e8), fontsize=8, color='gray')
ax.annotate('7 nm', xy=(2019, 1e10), fontsize=8, color='gray')
ax.annotate('3 nm', xy=(2023, 2e11), fontsize=8, color='gray')

plt.tight_layout()
plt.show()

print("Moore's Law (1965): The number of transistors on a chip doubles approximately every 2 years.")
print(f'\nFrom 1971 to 2023: {counts[-1]/counts[0]:.0e}x increase in transistors.')
print(f'That is roughly 2^{np.log2(counts[-1]/counts[0]):.1f} doublings in {2023-1971} years.')
print(f'Actual rate: one doubling every {(2023-1971)/np.log2(counts[-1]/counts[0]):.1f} years.')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Interactive-style exploration: CMOS inverter with different supply voltages

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

supply_voltages = [3.3, 5.0, 12.0]

for ax, VCC in zip(axes, supply_voltages):
    Vin = np.linspace(0, VCC, 1000)
    
    # Transfer characteristic (tanh model)
    midpoint = VCC / 2
    steepness = 8.0 / VCC
    Vout = VCC / 2 * (1 - np.tanh(steepness * (Vin - midpoint)))
    
    ax.plot(Vin, Vout, 'b-', linewidth=2.5)
    
    # Mark logic thresholds (70% / 30% of VCC for CMOS)
    VIH = 0.7 * VCC
    VIL = 0.3 * VCC
    VOH = 0.9 * VCC  # approximate
    VOL = 0.1 * VCC
    
    ax.axhline(y=VOH, color='green', linestyle='--', alpha=0.5)
    ax.axhline(y=VOL, color='red', linestyle='--', alpha=0.5)
    ax.axvline(x=VIH, color='orange', linestyle=':', alpha=0.5)
    ax.axvline(x=VIL, color='purple', linestyle=':', alpha=0.5)
    
    ax.text(VCC * 0.05, VOH + 0.1, f'VOH={VOH:.1f}V', fontsize=8, color='green')
    ax.text(VCC * 0.05, VOL - 0.3, f'VOL={VOL:.1f}V', fontsize=8, color='red')
    
    # Noise margin
    NMH = VOH - VIH
    NML = VIL - VOL
    
    ax.set_xlabel('Vin (V)', fontsize=11)
    ax.set_ylabel('Vout (V)', fontsize=11)
    ax.set_title(f'VCC = {VCC}V\nNM_H={NMH:.1f}V, NM_L={NML:.1f}V', fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, VCC)
    ax.set_ylim(-0.3, VCC + 0.3)
    ax.plot([0, VCC], [VCC, 0], 'k:', alpha=0.2, label='Unity gain line')

fig.suptitle('CMOS Inverter Transfer Curve at Different Supply Voltages',
             fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print('Higher VCC gives larger noise margins (more room for noise before logic errors).')
print('But higher VCC also means more dynamic power: P = C * VCC^2 * f')
print('This is the fundamental trade-off that drives the industry toward lower voltages.')

## Experiment — Build a CMOS Inverter on Breadboard

### Parts needed
- 1x 2N7000 NMOS transistor (TO-92 package: S-G-D pinout)
- 1x AO3401 PMOS transistor (SOT-23 -- you may need a breakout board, or use an IRF9540 in TO-220)
- Bench power supply set to 5V
- 10k potentiometer (for sweeping input voltage)
- Klein MM300 multimeter
- Breadboard + jumper wires

### Circuit Diagram

```
   +5V (Bench Supply)
    |
    +--- PMOS Source (S)
    |                                      Klein MM300
    |    PMOS Gate (G) ---+--- IN          (DC Voltage)
    |                     |                    |
    +--- PMOS Drain (D) --+--- OUT ---[test point]---+
    |                     |                          |
    |    NMOS Drain (D) --+                          |
    |                                                |
    |    NMOS Gate (G) ---+--- IN                    |
    |                     |                          |
    +--- NMOS Source (S)  |                     GND--+
    |                     |
   GND              Potentiometer
                    (wiper = IN)
                    between +5V and GND
```

### Procedure

1. **Wire the power rails**: +5V and GND from bench supply to breadboard rails.

2. **Place the PMOS**: Source to +5V rail. Gate and drain will be connected in step 4.

3. **Place the NMOS (2N7000)**: Source to GND rail. Pinout is S-G-D (flat side facing you, left to right).

4. **Connect the circuit**:
   - PMOS drain to NMOS drain -- this is the OUTPUT node
   - PMOS gate to NMOS gate -- this is the INPUT node
   - Connect the potentiometer as a voltage divider: one end to +5V, other to GND, wiper to INPUT

5. **Measure the transfer characteristic**:
   - Set multimeter to DC voltage
   - Slowly turn the pot from one end to the other
   - At several positions, measure both Vin (pot wiper to GND) and Vout (output to GND)
   - Record at least 10 data points

### Expected Results

| Vin (V) | Expected Vout (V) | Reason |
|---------|-------------------|--------|
| 0.0 | ~4.8-5.0 | PMOS fully ON, NMOS OFF |
| 1.0 | ~4.8-5.0 | Below NMOS threshold |
| 2.0 | ~4.0-4.5 | NMOS starting to turn on |
| 2.5 | ~2.5 | Transition region |
| 3.0 | ~0.5-1.0 | NMOS mostly on |
| 4.0 | ~0.1-0.2 | NMOS fully on |
| 5.0 | ~0.01-0.05 | NMOS fully on, PMOS OFF |

The 2N7000 has Vth ~ 2.1V, so the transition will be somewhat above VCC/2.

### What to observe
- The transition is **sharp** -- a small change in Vin causes a large change in Vout
- In the stable states, output is very close to the supply rails (rail-to-rail output)
- There is essentially **no current draw** in the stable states (measure supply current!)

## Experiment — Build a NAND Gate from Discrete MOSFETs

### Additional parts needed
- 1 more 2N7000 NMOS (2 total)
- 1 more PMOS (2 total)

### Circuit Diagram

```
       +5V                    +5V
        |                      |
   PMOS P1 (S)            PMOS P2 (S)
        |                      |
  A --- P1 (G)           B --- P2 (G)
        |                      |
   PMOS P1 (D) ----+---- PMOS P2 (D)
                    |
                    +-------- OUT
                    |
              NMOS N1 (D)
                    |
              A --- N1 (G)
                    |
              NMOS N1 (S)
                    |
              NMOS N2 (D)
                    |
              B --- N2 (G)
                    |
              NMOS N2 (S)
                    |
                   GND
```

### Procedure

1. Build the circuit as shown. **PMOS in parallel** (sources to VCC, drains tied together at output). **NMOS in series** (N1 drain to output, N1 source to N2 drain, N2 source to GND).

2. Use jumper wires to set inputs A and B to either +5V (HIGH) or GND (LOW).

3. For each combination, measure Vout with the multimeter.

### Expected Results

| A | B | Expected Vout | Measured |
|---|---|--------------|----------|
| GND | GND | ~5.0V (HIGH) | ________ |
| GND | +5V | ~5.0V (HIGH) | ________ |
| +5V | GND | ~5.0V (HIGH) | ________ |
| +5V | +5V | ~0.0V (LOW) | ________ |

This is the NAND truth table. You have built the fundamental building block of all digital logic from discrete transistors.

## Simulation — Falstad Circuit Simulator

### CMOS Inverter

Open this link to explore a CMOS inverter interactively:

[Falstad CMOS Inverter](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgpABZsKBTAWjDACgA3EFFcDQiB58eUCGzYUMWCGiYV8AM3Y5OEHljBDkPHJFwS5UkHxAaUYfEJxrsUFIaA)

Things to try:
- Toggle the input switch and watch the output
- Observe which transistor is conducting (highlighted green)
- Notice the brief moment during switching when both transistors conduct

### CMOS NAND Gate

[Falstad CMOS NAND Gate](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgpABZsKBTAWjDACgA3cFbKkPMJD58oENmwoYaWFBDRMK+AGbscnCDyxgh0kDkiqp0qJFzTkRsJGn6QfEOpQV8m3Fw5A)

Things to try:
- Toggle inputs A and B independently
- Verify the NAND truth table
- Watch the current flow path change with each input combination
- Notice: there is NEVER a direct path from VCC to GND in any stable state

## Datasheet Connection

The concepts in this notebook connect directly to IC datasheets:

### 1. Logic Threshold Voltages
Every digital IC datasheet specifies:
- **V_IH** (input HIGH threshold): minimum voltage recognized as logic 1
- **V_IL** (input LOW threshold): maximum voltage recognized as logic 0
- **V_OH** (output HIGH voltage): guaranteed minimum output for logic 1
- **V_OL** (output LOW voltage): guaranteed maximum output for logic 0

These come directly from the CMOS inverter transfer characteristic we plotted.

### 2. Quiescent Current (I_CC)
CMOS datasheets specify incredibly low static current:
```
74HC00 (CMOS NAND): I_CC = 4 uA typical at 5V
```
This is because CMOS has no static power path. Compare to TTL:
```
74LS00 (TTL NAND):  I_CC = 1.6 mA typical
```
That is 400x more static power! This is the direct consequence of the CMOS complementary structure.

### 3. Propagation Delay
```
74HC00: t_PHL = 7 ns, t_PLH = 7 ns (at VCC=5V)
```
This delay comes from charging/discharging the gate capacitances of the next stage, which is fundamentally limited by the MOSFET switching speed.

### 4. Output Drive Current
```
74HC00: I_OH = -4 mA, I_OL = 4 mA
```
These tell you how much current the PMOS (sourcing) and NMOS (sinking) can deliver. The NMOS can typically sink more current because electrons have higher mobility than holes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Compare CMOS vs TTL power consumption vs frequency

freq = np.logspace(0, 8, 200)  # 1 Hz to 100 MHz

# CMOS (74HC00): P = P_static + C_PD * VCC^2 * f
VCC = 5.0
P_static_cmos = 4e-6 * VCC  # 4 uA * 5V = 20 uW
C_PD_cmos = 22e-12  # 22 pF typical
P_dynamic_cmos = C_PD_cmos * VCC**2 * freq
P_total_cmos = P_static_cmos + P_dynamic_cmos

# TTL (74LS00): roughly constant power
P_ttl = np.ones_like(freq) * 1.6e-3 * VCC  # 1.6 mA * 5V = 8 mW
# TTL also has some frequency-dependent component but it is smaller
P_ttl_dynamic = 10e-12 * VCC**2 * freq  # smaller C_PD
P_total_ttl = P_ttl + P_ttl_dynamic

fig, ax = plt.subplots(figsize=(10, 6))

ax.loglog(freq, P_total_cmos * 1000, 'b-', linewidth=2.5, label='CMOS (74HC00)')
ax.loglog(freq, P_total_ttl * 1000, 'r-', linewidth=2.5, label='TTL (74LS00)')

# Mark crossover point
crossover_idx = np.argmin(np.abs(P_total_cmos - P_total_ttl))
ax.plot(freq[crossover_idx], P_total_cmos[crossover_idx] * 1000, 'ko', markersize=10)
ax.annotate(f'Crossover\n~{freq[crossover_idx]:.0e} Hz',
            xy=(freq[crossover_idx], P_total_cmos[crossover_idx] * 1000),
            xytext=(freq[crossover_idx] * 5, P_total_cmos[crossover_idx] * 1000 * 5),
            fontsize=10, arrowprops=dict(arrowstyle='->', lw=1.5))

ax.set_xlabel('Switching Frequency (Hz)', fontsize=12)
ax.set_ylabel('Power Consumption (mW)', fontsize=12)
ax.set_title('CMOS vs TTL Power Consumption\n(Per Gate, VCC = 5V)', fontsize=13, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, which='both')

ax.fill_between(freq, P_total_cmos * 1000, P_total_ttl * 1000,
                where=P_total_cmos < P_total_ttl,
                alpha=0.15, color='blue', label='CMOS advantage')
ax.fill_between(freq, P_total_cmos * 1000, P_total_ttl * 1000,
                where=P_total_cmos > P_total_ttl,
                alpha=0.15, color='red', label='TTL advantage')

plt.tight_layout()
plt.show()

print('At low frequencies, CMOS wins dramatically (near-zero static power).')
print('At very high frequencies, CMOS dynamic power dominates.')
print(f'Crossover is around {freq[crossover_idx]:.0e} Hz.')
print('For most practical applications, CMOS is the clear winner -- which is why CMOS won.')

## Checkpoint Questions

Test your understanding before moving on:

---

**Q1:** In a CMOS inverter, when the input is HIGH (VCC), which transistor is ON and which is OFF? What voltage appears at the output?

<details>
<summary>Answer</summary>

When input = HIGH (VCC): the NMOS is ON (Vgs = VCC > Vth) and the PMOS is OFF (Vgs = 0). The output is connected to GND through the NMOS, so output = LOW (~0V). The inverter inverts: HIGH in, LOW out.

</details>

---

**Q2:** Why does a CMOS gate consume almost zero power in a static (non-switching) state? What changes when the gate switches rapidly?

<details>
<summary>Answer</summary>

In any stable state, one transistor in the complementary pair is ON and the other is OFF. There is never a DC path from VCC to GND, so static current is essentially zero (only tiny leakage current flows). When the gate switches, it must charge and discharge the load capacitance, which requires current from the supply. The dynamic power is P = C * VCC^2 * f, which increases linearly with switching frequency.

</details>

---

**Q3:** In a CMOS NAND gate, the PMOS transistors are in parallel and the NMOS transistors are in series. Why this arrangement? What would happen if you swapped them?

<details>
<summary>Answer</summary>

For a NAND gate, the output should be LOW only when ALL inputs are HIGH. NMOS in series means both must be ON (both inputs HIGH) to pull the output to GND. PMOS in parallel means any single LOW input turns on a PMOS, pulling the output HIGH. If you swapped them (PMOS in series, NMOS in parallel), you would get a NOR gate instead: output LOW when ANY input is HIGH.

</details>

---

**Q4:** A 2N7000 NMOS has a threshold voltage Vth of about 2.1V. If you build a CMOS inverter with VCC = 3.3V, approximately where would you expect the switching threshold to be? Would this be a good logic gate?

<details>
<summary>Answer</summary>

The switching threshold of a CMOS inverter is typically near VCC/2, but it shifts based on the relative strengths and thresholds of the NMOS and PMOS. With Vth_NMOS = 2.1V and VCC = 3.3V, the NMOS only has 1.2V of overdrive (VCC - Vth). This means the transition region would be shifted high and the gate would have asymmetric noise margins. It would work but with reduced noise margin on the HIGH side. For 3.3V logic, you would want MOSFETs with lower thresholds (~0.7V for logic-level devices).

</details>

---

**Q5:** Moore's Law says transistor count doubles every ~2 years. If a chip had 1 billion transistors in 2010, approximately how many would you expect in 2024?

<details>
<summary>Answer</summary>

From 2010 to 2024 is 14 years, which is 7 doubling periods. 1 billion x 2^7 = 1 billion x 128 = approximately 128 billion transistors. This is quite close to actual modern chips like Apple's M2 Ultra (134 billion transistors), confirming that Moore's Law has held remarkably well.

</details>

---

**Q6:** Why is NAND preferred over NOR for building complex logic, even though both are universal gates?

<details>
<summary>Answer</summary>

In CMOS, PMOS transistors are inherently slower than NMOS because hole mobility is lower than electron mobility. In a NAND gate, the slower PMOS devices are in parallel (which is faster than series), while the faster NMOS are in series. In a NOR gate, the arrangement is reversed: slow PMOS in series (worst case) and fast NMOS in parallel. This makes NAND gates faster. Additionally, most logic functions require fewer NAND gates than NOR gates to implement.

</details>

---

## Summary

| Concept | Key Takeaway |
|---------|--------------|
| Digital logic | Simplify voltages to just HIGH (1) and LOW (0) for noise immunity |
| CMOS inverter | Complementary NMOS + PMOS, no static power consumption |
| NAND gate | 2 NMOS series + 2 PMOS parallel, output LOW only when both inputs HIGH |
| NOR gate | 2 PMOS series + 2 NMOS parallel, output HIGH only when both inputs LOW |
| NAND universality | Any logic function can be built from NAND gates alone |
| Why NAND wins | PMOS in parallel is faster; fewer gates needed for most functions |
| Moore's Law | Transistor density doubles ~every 2 years, enabled by CMOS scaling |

**The key insight**: The same MOSFET physics from Module 04 (threshold voltage, gate capacitance, complementary N/P types) directly enables all of digital logic. Every gate in every processor is fundamentally a CMOS transistor pair.

**Next up**: [01 -- Logic Families and Voltage Levels](./01-logic-families-and-levels.ipynb) -- standardized logic families, TTL vs CMOS, and why voltage levels matter.